In [12]:
import sys
sys.path.append("..") 
from utils.surface_gases import *

In [13]:
COUPLER_options = {}

COUPLER_options['Phi_global'] = 1.0
COUPLER_options['T_outgas'] = 3400.0

COUPLER_options['radius'] = 6.371e6
COUPLER_options['mass']   = 5.972e24
COUPLER_options['gravity'] = 6.67e-11 * COUPLER_options['mass'] / (COUPLER_options['radius']**2)
COUPLER_options['hydrogen_earth_oceans'] = 10.0
COUPLER_options['CH_ratio'] = 2.0
COUPLER_options['fO2_shift_IW'] = 5.0
COUPLER_options['nitrogen_ppmw'] = 2.0
COUPLER_options['planet_coresize'] = 0.55

solvevol_vols = ['H2O', 'CO2', 'N2', 'H2', 'CO', 'CH4']
for v in solvevol_vols:
    COUPLER_options[v+"_included"] = 0
    COUPLER_options[v+"_initial_bar"] = 0.0

COUPLER_options["H2O_included"] = 1
COUPLER_options["CO2_included"] = 1
COUPLER_options["N2_included"]  = 1
COUPLER_options["H2_included"]  = 1
COUPLER_options["CO_included"]  = 1
COUPLER_options["CH4_included"] = 1

# COUPLER_options["H2O_initial_bar"] = 1.0
# COUPLER_options["CO2_initial_bar"] = 0.0
# COUPLER_options["N2_initial_bar"]  = 0.0



In [14]:
COUPLER_options["mantle_mass"] = calc_mantle_mass(COUPLER_options)

In [15]:
solvevol_target = solvevol_get_target_from_params(COUPLER_options)
# solvevol_target = solvevol_get_target_from_pressures(COUPLER_options)

print("Targets:")
print(solvevol_target)

Targets:
{'H': 1.5499999999999997e+21, 'C': 3.0999999999999995e+21, 'N': 8.061672162064889e+18}


In [16]:
p_d = solvevol_equilibrium_atmosphere(solvevol_target, COUPLER_options)

print(p_d)

p_bar = {}
for v in volatile_species:
    p_bar[v] = p_d[v+"_atm_bar"]

mubar = 0.0
ptot = 0.0
print("Partial pressures")
for v in solvevol_vols:
    print("    %s : %.3f bar"%(v, p_bar[v]))
    ptot += p_bar[v]
    mubar += p_bar[v]*molar_mass[v]
mubar /= ptot

print("mubar =",mubar)

m_d = {}
for v in solvevol_vols:
    m_d[v] = p_bar[v]*1.0e5 * 4 * 3.14 * (molar_mass[v]/mubar) * COUPLER_options["radius"]**2.0 / COUPLER_options["gravity"]

# print(solvevol_atmosphere_mass(p_bar, COUPLER_options))

print(" ")
print(m_d)

H_kg = (2*m_d["H2O"]/molar_mass["H2O"] + 2*m_d["H2"]/molar_mass["H2"] + 4*m_d["CH4"]/molar_mass["CH4"]) *molar_mass['H']
C_kg = (m_d["CO2"]/molar_mass["CO2"] + m_d["CO"]/molar_mass["CO"] + m_d["CH4"]/molar_mass["CH4"]) * molar_mass['C']
N_kg = m_d["N2"]

print(H_kg, C_kg, N_kg)


{'M_atm': 8.700977542121491e+21, 'P_surf': 1674.0691619527001, 'H2O_atm_bar': 42.41567847384332, 'H2O_atm_kg': 9.420653903841033e+19, 'H2O_liquid_kg': 1.3755919749008847e+22, 'H2O_solid_kg': 0.0, 'H2O_total_kg': 1.3850126288047256e+22, 'CO2_atm_bar': 1507.465213208285, 'CO2_atm_kg': 8.179227382856643e+21, 'CO2_liquid_kg': 2.4571717861595294e+21, 'CO2_solid_kg': 0.0, 'CO2_total_kg': 1.0636399169016173e+22, 'H2_atm_bar': 0.4085679530842879, 'H2_atm_kg': 1.0154126958426656e+17, 'H2_liquid_kg': 0.0, 'H2_solid_kg': 0.0, 'H2_total_kg': 1.0154126958426656e+17, 'CH4_atm_bar': 5.84751982412301e-11, 'CH4_atm_kg': 115635100.17823114, 'CH4_liquid_kg': 3261554.231381783, 'CH4_solid_kg': 0.0, 'CH4_total_kg': 118896654.40961292, 'CO_atm_bar': 121.55131885544102, 'CO_atm_kg': 4.197458378423871e+20, 'CO_liquid_kg': 4.0047485297325105e+19, 'CO_solid_kg': 0.0, 'CO_total_kg': 4.597933231397122e+20, 'N2_atm_bar': 2.2283834619881557, 'N2_atm_kg': 7.696241114350808e+18, 'N2_liquid_kg': 3.654310477089985e+17,

/home/n/nichollsh/PROTEUS/tools/../utils/surface_gases.py:76: RuntimeWarning: invalid value encountered in scalar power
  return const*p**exponent
/home/n/nichollsh/PROTEUS/tools/../utils/surface_gases.py:149: RuntimeWarning: invalid value encountered in scalar power
  ppmw  = pb_N2**0.5 * np.exp(5908.0 * pb_tot**0.5/temp - 1.6*fO2_shift)


In [17]:
H_mol = H_kg / (1.0e-3)
H_oc  = H_mol / (7.69e22 * 2.0)


print(H_oc)
print("C/H = %.4f" % (C_kg/H_kg))

0.06916992604947114
C/H = 226.6337
